In [ ]:
from sodapy import Socrata
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from config import apiKey

from urllib.request import urlopen
from scipy import stats
import gmaps
import plotly.express as px



# Use open client to obtain data
client = Socrata("data.cdc.gov", apiKey)
results = client.get("dnhi-s2bf", limit = 1000000)

# Convert to pandas DataFrame
death_quarterly = pd.DataFrame.from_records(results)
death_quarterly.head()


In [ ]:
death_quarterly_1 = death_quarterly.drop(columns=['footnote'])
death_quarterly_2 = death_quarterly_1.dropna(subset =['covid_19_deaths','total_deaths'])


In [ ]:
death_quarterly_2['total_deaths'] = death_quarterly_2['total_deaths'].astype(int)
death_quarterly_2['covid_19_deaths'] = death_quarterly_2['covid_19_deaths'].astype(int)
death_gb = death_quarterly_2.groupby(['fips_code'])
death_gb_df = death_gb.sum()

In [ ]:
death_gb_df['covid_portion_of_total_deaths'] = death_gb_df['covid_19_deaths']/death_gb_df['total_deaths']
death_gb_df = pd.DataFrame(death_gb_df)
death_gb_df.reset_index(inplace=True)


In [ ]:
death_gb_df

In [ ]:
death_gb_df.reset_index(inplace=True)

In [ ]:
death_gb_df.describe(include='all')

In [ ]:
from urllib.request import urlopen
import json
import requests
from scipy import stats
import pandas as pd
import numpy as np
import gmaps
from scipy.stats import linregress
from matplotlib import pyplot as plt
import plotly.express as px


url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
response = requests.get(url).json()
response

In [ ]:
death_gb_df.head()

In [ ]:
fig = px.choropleth(death_gb_df, geojson=response, locations='fips_code', color='covid_portion_of_total_deaths',
                           color_continuous_scale="ylgnbu",
                           range_color=(0, .35),
                           scope="usa",
                           labels={'covid_portion_of_total_deaths':'Covid Deaths as % of Total Deaths'}
                          )
fig.show()

In [ ]:
death_gb_df.describe()
death_df = death_gb_df.rename(columns = {'fips_code':'fips'})
death_df.head()

In [ ]:
# Use open client to obtain data
client = Socrata("data.cdc.gov", apiKey)
results = client.get("8xkx-amqh", limit = 2000000)
                     
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head()

In [ ]:
results_df_columns = results_df[['date','fips','recip_state','recip_county',
                                     'completeness_pct','administered_dose1_recip',
                                     'administered_dose1_pop_pct','series_complete_yes',
                                     'series_complete_pop_pct','metro_status','census2019']]

In [ ]:
#restart from here

In [ ]:
clean_df = results_df_columns.dropna()
clean_df['date'] = pd.to_datetime(clean_df['date']).dt.date

In [ ]:
clean_df['month'] = pd.DatetimeIndex(clean_df['date']).month

In [ ]:
clean_df['quarter'] = pd.DatetimeIndex(clean_df['date']).quarter
clean_df['year'] = pd.DatetimeIndex(clean_df['date']).year
clean_df.head()

In [ ]:
grouped_fips = clean_df.groupby(['fips'])
max_group = grouped_fips.max()

In [ ]:
max_dose = pd.DataFrame(max_group[['administered_dose1_recip','administered_dose1_pop_pct']]) 
max_dose.head()

In [ ]:
merged_df = death_df.merge(max_dose,on='fips')
merged_df.head()

In [ ]:
x_axis = merged_df['administered_dose1_pop_pct']
y_axis = merged_df['covid_portion_of_total_deaths']

plt.scatter(x_axis,y_axis)
plt.show()

In [ ]:
death_quarterly_2 = death_quarterly.rename(columns = {'fips_code' : 'fips'})
death_quarterly_2['year']= death_quarterly_2['year'].astype(int)
death_quarterly_2['quarter']=death_quarterly_2['quarter'].astype(int)
clean_df['year']=clean_df['year'].astype(int)
clean_df['quarter']=clean_df['quarter'].astype(int)

In [ ]:
len(clean_df)

In [ ]:
total_quarter = pd.merge(death_quarterly_2,clean_df, on = ('fips','year','quarter'),how ='inner')
total_quarter.head()
total_quarter_2 = total_quarter.loc[(total_quarter['state_of_residence']=='Florida')|(total_quarter['state_of_residence'] =='Texas')|(total_quarter['state_of_residence'] =='California')|(total_quarter['state_of_residence'] =='New York'),:]
total_quarter_2.head()
len(total_quarter)

In [ ]:
total_quarter_2['quarter_number'] = pd.to_datetime(total_quarter_2['year'].astype(str) + 'Q' + total_quarter_2['quarter'].astype(str))
url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"

response = requests.get(url).json()
response

In [ ]:
total_quarter_2.head()

In [ ]:
# x_axis= total_quarter_2['quarter_number']
# y_axis= total_quarter_2['']


In [ ]:
total_quarter_2.dropna(subset = ['covid_19_deaths','total_deaths'])
total_quarter_2['covid_death_rate']= total_quarter_2['covid_19_deaths'].astype(float)/total_quarter_2['total_deaths'].astype(float)
total_quarter_2.head(25)


In [ ]:
len(total_quarter_2)

In [ ]:
total_quarter_2.drop_duplicates(subset =['quarter_number','fips'],inplace=True)
                                                                                                                                       

In [ ]:
total_quarter_2.head()

In [ ]:
len(total_quarter_2)
# fig = px.choropleth(death_gb_df, geojson=response, locations='fips_code', color='covid_portion_of_total_deaths',
#                            color_continuous_scale="bluered",
#                            range_color=(0.1, .25),
#                            scope="usa",
#                            labels={'COVID as % of Total Deaths':'covid_portion_of_total_deaths'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
# x_axis= total_quarter_2['quarter_number']
# y_axis= total_quarter_2['']


In [ ]:
total_quarter_2.columns

In [ ]:
total_quarter_2['administered_dose1_recip'] = total_quarter_2['administered_dose1_recip'].astype(float) 
total_quarter_2['census2019'] = total_quarter_2['census2019'].astype(float) 
total_quarter_2['covid_19_deaths'] = total_quarter_2['covid_19_deaths'].astype(float) 
total_quarter_2['total_deaths'] = total_quarter_2['total_deaths'].astype(float) 



In [ ]:
california_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'California',:]
california_df.head()

cali_quarter_group = california_df.groupby('quarter_number')
cali_quarter_df = cali_quarter_group.sum()


In [ ]:
cali_quarter_df.head()

In [ ]:
cali_quarter_df['covid_death_rate']= cali_quarter_df['covid_19_deaths']/cali_quarter_df['total_deaths']
cali_quarter_df['vaccine_rate']= cali_quarter_df['administered_dose1_recip']/cali_quarter_df['census2019']

cali_quarter_df.head()

In [ ]:
x_axis = cali_quarter_df.index
y_axis = cali_quarter_df['vaccine_rate']

plt.plot(x_axis, y_axis)
plt.title("Title")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()


In [ ]:
total_quarter['administered_dose1_recip'] = total_quarter['administered_dose1_recip'].astype(float) 
total_quarter['census2019'] = total_quarter['census2019'].astype(float) 
total_quarter['covid_19_deaths'] = total_quarter['covid_19_deaths'].astype(float) 
total_quarter['total_deaths'] = total_quarter['total_deaths'].astype(float) 



In [ ]:
florida_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Florida',:]
florida_df.head()

FL_quarter_group = florida_df.groupby('quarter_number')
FL_quarter_df = FL_quarter_group.sum()

In [ ]:
FL_quarter_df['covid_death_rate']= FL_quarter_df['covid_19_deaths']/FL_quarter_df['total_deaths']
FL_quarter_df['vaccine_rate']= FL_quarter_df['administered_dose1_recip']/FL_quarter_df['census2019']

FL_quarter_df.head()

In [ ]:
x2_axis = FL_quarter_df.index
y2_axis = FL_quarter_df['vaccine_rate']

plt.plot(x2_axis, y2_axis)
plt.title("Title")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
texas_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'Texas',:]
texas_df.head()

TX_quarter_group = texas_df.groupby('quarter_number')
TX_quarter_df = TX_quarter_group.sum()
TX_quarter_df.head()

In [ ]:
TX_quarter_df['covid_death_rate']= TX_quarter_df['covid_19_deaths']/TX_quarter_df['total_deaths']
TX_quarter_df['vaccine_rate']= TX_quarter_df['administered_dose1_recip']/TX_quarter_df['census2019']

TX_quarter_df.head()

In [ ]:
x3_axis = TX_quarter_df.index
y3_axis = TX_quarter_df['covid_19_deaths']/TX_quarter_df['total_deaths']

plt.plot(x3_axis, y3_axis)
plt.title("Title")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
(TX_quarter_df['covid_death_rate']*100).sort_index().plot.line()

In [ ]:
state_CAFL_df = total_quarter_2.loc[(total_quarter_2['state_of_residence'] == 'California')|(total_quarter_2['state_of_residence'] == 'Florida')|(total_quarter_2['state_of_residence'] == 'New York')|(total_quarter_2['state_of_residence'] == 'Texas'),:]
state_CAFL_df.head()

In [ ]:
CAFL_quarter_group = state_CAFL_df.groupby(['state_of_residence','quarter_number'])
CAFL_quarter_df = CAFL_quarter_group.sum()

CAFL_quarter_df['covid_death_rate']= CAFL_quarter_df['covid_19_deaths']/CAFL_quarter_df['total_deaths']
CAFL_quarter_df['vaccine_rate']= CAFL_quarter_df['administered_dose1_recip']/CAFL_quarter_df['census2019']

CAFL_quarter_df.head()

In [ ]:
cali_quarter_df.plot(use_index=True, y=["covid_death_rate", "vaccine_rate"], kind="bar")

In [ ]:
FL_quarter_df.plot(use_index=True, y=["covid_death_rate", "vaccine_rate"], kind="bar")

In [ ]:
len(total_quarter)

In [ ]:
scatter_quarter = total_quarter

scatter_quarter['quarter_number'] = pd.to_datetime(total_quarter['year'].astype(str) + 'Q' + total_quarter['quarter'].astype(str))
scatter_quarter.head()

In [ ]:
#lag timeseries regression
scatter_plot = scatter_quarter.loc[:,['fips','administered_dose1_recip','census2019','covid_19_deaths','total_deaths','quarter_number']]
scatter_plot.head()

In [ ]:
len(scatter_plot)

In [ ]:
scatter_plot.drop_duplicates(subset =['quarter_number','fips'],inplace=True)

In [ ]:
len(scatter_plot)

In [ ]:
scatter_plot.dropna()

In [ ]:
clean_scatter = scatter_plot.loc[(scatter_plot['administered_dose1_recip'] != 0) & (scatter_plot['covid_19_deaths'] != 0) & (scatter_plot['census2019'] != 0) &(scatter_plot['total_deaths'] != 0),:]
len(clean_scatter)

In [ ]:
xx = scatter_plot['administered_dose1_recip']/scatter_plot['census2019']
yy = scatter_plot['covid_19_deaths']/scatter_plot['total_deaths']

fig1, ax1 = plt.subplots()
plt.scatter(xx,yy)
plt.xlabel('Vaccine %')
plt.ylabel('Covid Death %')
plt.show()

In [ ]:
clean_scatter.replace([np.inf, -np.inf], np.nan, inplace=True)
clean_scatter.dropna(inplace=True)
len(clean_scatter)

In [ ]:
x_x = clean_scatter['administered_dose1_recip']/clean_scatter['census2019']
y_y = clean_scatter['covid_19_deaths']/clean_scatter['total_deaths']

fig1, ax1 = plt.subplots()
plt.scatter(x_x,y_y)
plt.xlabel('Vaccine %')
plt.ylabel('Covid Death %')
plt.show()

In [ ]:
correlation = stats.pearsonr(x_x,y_y)
print(f"The correlation between vaccine percent \033[34mand\033[0m the covid death percent \033[34mis\033[0m \033[31m{round(correlation[0],2)}\033[0m")

# Creating Plot for scatter and adding line
(slope, intercept,rvalue, pvalue, stderr) = linregress(x_x,y_y)
regress = x_x* slope + intercept
line_eq = f"y = {round(slope, 2)} x + {round(intercept, 2)}"

plt.scatter(x_x,y_y, s = 3)
plt.plot(x_x, regress, color='red')
plt.annotate(line_eq,(.4,.5),fontsize=12,color="red")
plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')

plt.savefig("../Images/Scatter_Vaccine_vs_Death")

plt.show()


In [ ]:
x_min = np.min(x_x)
x_max = np.max(x_x)
  
y_min = np.min(y_y)
y_max = np.max(y_y)

x_bins = np.linspace(x_min, x_max, 70)
y_bins = np.linspace(y_min, y_max, 50)

#plt.hexbin(x_x, y_y, bins = 150)

fig, ax = plt.subplots()
plt.hist2d(x_x, y_y, bins = [x_bins,y_bins],cmap = plt.cm.nipy_spectral)
#plt.tight_layout() 
plt.xlabel('Fraction of Pop Vaccinated')
plt.ylabel('Covid Death %')
plt.savefig("../Images/Scatter_Vaccine_vs_Death_Heat")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

xr = x_x.values.reshape(1, -1)

x_train,x_test,y_train,y_test = train_test_split(xr,y_y,test_size=0.3,random_state=0)

log_regression = LogisticRegression()
log_regression.fit(x_train,y_train)
y_pred = log_regression.predict(x_test)

cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

y_pred_proba = log_regression.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)

#create ROC curve
plt.plot(fpr,tpr,label="AUC="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
fit = np.polyfit(np.log(x_x), y_y, 1)
print(fit)

In [ ]:
y = a + b*ln(x)

In [ ]:
NY_df = total_quarter_2.loc[total_quarter_2['state_of_residence'] == 'New York',:]
NY_df.head()

NY_quarter_group = NY_df.groupby('quarter_number')
NY_quarter_df = NY_quarter_group.sum()

In [ ]:
NY_quarter_df['covid_death_rate']= NY_quarter_df['covid_19_deaths']/NY_quarter_df['total_deaths']
NY_quarter_df['vaccine_rate']= NY_quarter_df['administered_dose1_recip']/NY_quarter_df['census2019']

NY_quarter_df.head()

In [ ]:
NY_quarter_df.plot(use_index=True, y=["covid_death_rate", "vaccine_rate"], kind="bar")

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20
# Plot COVID-19 death counts by State grouped by Age Group
fig, ax = plt.subplots()
ny = ax.bar(index,NY_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,NY_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)
# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('New York COVID-19 Death Rate vs.  Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()
# Display the chart, and export for later use
plt.tight_layout()
plt.savefig('../Images/NY_Covid_Death_vs_Vaccine_Rate.png')
plt.show()

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20
# Plot COVID-19 death counts by State grouped by Age Group
fig, ax = plt.subplots()
ny = ax.bar(index,cali_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,cali_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)
# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('California COVID-19 Death Rate vs.  Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()
# Display the chart, and export for later use
plt.tight_layout()
plt.savefig('../Images/CA_Covid_Death_vs_Vaccine_Rate.png')
plt.show()

In [ ]:
# Set index length and bar width
quarter_label = ['2020 Q4', '2021 Q1','2021 Q2']
index = np.arange(3)
bar_width = 0.20
# Plot COVID-19 death counts by State grouped by Age Group
fig, ax = plt.subplots()
ny = ax.bar(index,FL_quarter_df["covid_death_rate"], bar_width, label='COVID-19 Death Rate', color='red', alpha=0.5)
ny2 = ax.bar(index+bar_width,FL_quarter_df["vaccine_rate"], bar_width, label='Vaccine % of Total', color='green', alpha=0.5)
# Assign title, labels, and legend
ax.set_xlabel('Quarter')
ax.set_ylabel('Percent')
ax.set_title('Florida COVID-19 Death Rate vs. Vaccine Percent')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(quarter_label, rotation=45, ha='right')
ax.legend()
# Display the chart, and export for later use
plt.tight_layout()
plt.savefig('../Images/FL_Covid_Death_vs_Vaccine_Rate.png')
plt.show()